# Pengantar Probabilitas dan Statistik
Dalam notebook ini, kita akan mencoba beberapa konsep yang telah kita bahas sebelumnya. Banyak konsep dari probabilitas dan statistik yang diwakili dengan baik dalam pustaka utama untuk pemrosesan data di Python, seperti `numpy` dan `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variabel Acak dan Distribusi
Mari kita mulai dengan mengambil sampel 30 nilai dari distribusi uniform dari 0 hingga 9. Kita juga akan menghitung mean dan varians.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Untuk memperkirakan secara visual berapa banyak nilai berbeda yang ada dalam sampel, kita bisa menggambar **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Menganalisis Data Nyata

Rata-rata dan varians sangat penting saat menganalisis data dunia nyata. Mari kita muat data tentang pemain bisbol dari [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Kami menggunakan paket yang disebut [**Pandas**](https://pandas.pydata.org/) di sini untuk analisis data. Kami akan membahas lebih lanjut tentang Pandas dan bekerja dengan data di Python nanti dalam kursus ini.

Mari kita hitung nilai rata-rata untuk usia, tinggi, dan berat:


In [ ]:
df[['Age','Height','Weight']].mean()

Sekarang mari kita fokus pada tinggi badan, dan hitung deviasi standar serta varians:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Selain nilai rata-rata, masuk akal untuk melihat nilai median dan kuartil. Mereka dapat divisualisasikan menggunakan **diagram kotak**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Kita juga dapat membuat plot kotak dari subset dataset kita, misalnya, dikelompokkan berdasarkan peran pemain.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Catatan**: Diagram ini menunjukkan, bahwa rata-rata, tinggi pemain base pertama lebih tinggi daripada tinggi pemain base kedua. Nanti kita akan belajar bagaimana kita dapat menguji hipotesis ini secara lebih formal, dan bagaimana menunjukkan bahwa data kita signifikan secara statistik untuk membuktikannya.  

Usia, tinggi, dan berat badan semuanya adalah variabel acak kontinu. Menurut Anda, bagaimana distribusinya? Cara yang baik untuk mengetahuinya adalah dengan memplot histogram nilai-nilai tersebut: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribusi Normal

Mari buat sampel buatan dari berat yang mengikuti distribusi normal dengan mean dan variansi yang sama seperti data nyata kita:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Karena sebagian besar nilai dalam kehidupan nyata terdistribusi secara normal, kita sebaiknya tidak menggunakan generator angka acak uniform untuk menghasilkan data sampel. Berikut ini yang terjadi jika kita mencoba menghasilkan berat dengan distribusi uniform (dihasilkan oleh `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Interval Kepercayaan

Sekarang mari kita hitung interval kepercayaan untuk berat dan tinggi pemain bisbol. Kita akan menggunakan kode [dari diskusi stackoverflow ini](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Pengujian Hipotesis

Mari kita jelajahi peran yang berbeda dalam dataset pemain baseball kita:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Mari kita uji hipotesis bahwa pemain First Basemen lebih tinggi daripada Second Basemen. Cara paling sederhana untuk melakukan ini adalah dengan menguji interval kepercayaan:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Kita dapat melihat bahwa intervalnya tidak tumpang tindih.

Cara yang secara statistik lebih benar untuk membuktikan hipotesis adalah dengan menggunakan **Student t-test**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dua nilai yang dikembalikan oleh fungsi `ttest_ind` adalah:
* p-value dapat dianggap sebagai probabilitas dari dua distribusi memiliki mean yang sama. Dalam kasus kami, nilainya sangat rendah, yang berarti ada bukti kuat yang mendukung bahwa pemain base pertama lebih tinggi.
* t-value adalah nilai antara dari perbedaan mean yang dinormalisasi yang digunakan dalam uji t, dan nilai ini dibandingkan dengan nilai ambang untuk nilai kepercayaan tertentu.


## Mensimulasikan Distribusi Normal dengan Teorema Limit Sentral

Generator pseudo-acak di Python dirancang untuk memberikan distribusi uniform. Jika kita ingin membuat generator untuk distribusi normal, kita dapat menggunakan teorema limit sentral. Untuk mendapatkan nilai yang terdistribusi normal, kita cukup menghitung rata-rata dari sampel yang dihasilkan secara uniform.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelasi dan Evil Baseball Corp

Korelasi memungkinkan kita menemukan hubungan antara urutan data. Dalam contoh mainan kita, mari kita berpura-pura ada sebuah perusahaan baseball jahat yang membayar pemainnya berdasarkan tinggi badan mereka - semakin tinggi pemain, semakin banyak uang yang dia dapatkan. Misalkan ada gaji pokok sebesar $1000, dan bonus tambahan dari $0 hingga $100, tergantung pada tinggi badan. Kita akan mengambil pemain nyata dari MLB, dan menghitung gaji imajinasi mereka:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Sekarang mari kita hitung kovarians dan korelasi dari urutan tersebut. `np.cov` akan memberi kita yang disebut **matriks kovarians**, yang merupakan perpanjangan dari kovarians ke banyak variabel. Elemen $M_{ij}$ dari matriks kovarians $M$ adalah korelasi antara variabel input $X_i$ dan $X_j$, dan nilai diagonal $M_{ii}$ adalah varians dari $X_{i}$. Demikian pula, `np.corrcoef` akan memberi kita **matriks korelasi**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelasi yang sama dengan 1 berarti ada **hubungan linier** yang kuat antara dua variabel. Kita dapat melihat hubungan linier secara visual dengan memplot satu nilai terhadap nilai lainnya:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Mari kita lihat apa yang terjadi jika relasinya tidak linier. Misalkan perusahaan kita memutuskan untuk menyembunyikan ketergantungan linier yang jelas antara tinggi badan dan gaji, dan memperkenalkan sedikit non-linieritas ke dalam rumus, seperti `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Dalam kasus ini, korelasinya sedikit lebih kecil, tetapi masih cukup tinggi. Sekarang, untuk membuat hubungan tersebut menjadi kurang jelas, kita mungkin ingin menambahkan sedikit keacakan ekstra dengan menambahkan beberapa variabel acak ke gaji. Mari kita lihat apa yang terjadi:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Bisakah Anda menebak mengapa titik-titik itu berbaris menjadi garis vertikal seperti ini?

Kita telah mengamati korelasi antara konsep rekayasa buatan seperti gaji dan variabel teramati *tinggi badan*. Mari kita juga lihat apakah kedua variabel teramati, seperti tinggi badan dan berat badan, juga berkorelasi:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Sayangnya, kami tidak mendapatkan hasil apa pun - hanya beberapa nilai `nan` yang aneh. Ini disebabkan oleh fakta bahwa beberapa nilai dalam deret kami tidak terdefinisi, yang diwakili sebagai `nan`, yang menyebabkan hasil operasi juga menjadi tidak terdefinisi. Dengan melihat matriks, kita dapat melihat bahwa `Weight` adalah kolom yang bermasalah, karena korelasi sendiri antara nilai `Height` telah dihitung.

> Contoh ini menunjukkan pentingnya **persiapan data** dan **pembersihan**. Tanpa data yang tepat, kita tidak dapat menghitung apa pun.

Mari kita gunakan metode `fillna` untuk mengisi nilai yang hilang, dan menghitung korelasinya: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Memang ada korelasi, tetapi tidak sekuat dalam contoh buatan kita. Memang, jika kita melihat plot sebar satu nilai terhadap yang lain, hubungan tersebut akan jauh kurang jelas:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Kesimpulan

Dalam notebook ini kita telah belajar bagaimana melakukan operasi dasar pada data untuk menghitung fungsi statistik. Kini kita tahu bagaimana menggunakan perangkat matematika dan statistik yang tepat untuk membuktikan beberapa hipotesis, serta bagaimana menghitung interval kepercayaan untuk variabel sembarang berdasarkan sampel data.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk mencapai ketepatan, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang sahih. Untuk informasi penting, disarankan menggunakan jasa terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau salah tafsir yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
